In [ ]:
%pip install webdriver_manager
%pip install selenium

In [7]:
# 시작 날짜와 종료 날짜 설정
start_date = datetime(2020, 1, 1)
end_date = datetime(2020, 12, 31)

# 총 일수 계산
delta = end_date - start_date

# 리스트 컴프리헨션으로 날짜 리스트 생성
dates = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(delta.days + 1)]

# 결과 확인
print(dates)

['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


options = webdriver.ChromeOptions()
prefs = {"download.default_directory": "./data/spotify_files/"}
options.add_experimental_option("prefs", prefs)


# Countries and dates
countries = ['kr']

start_date = datetime(2021, 2, 4)
end_date = datetime(2024, 10, 1)

delta = end_date - start_date
dates = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(0, delta.days + 1, 7)]


with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    driver.get("https://charts.spotify.com/charts/view/regional-global-weekly/latest")
    
    # Handle cookie banner if present
    try:
        accept_cookies = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
        )
        accept_cookies.click()
    except:
        pass  # If not present, continue

    # Wait for the login button and click it
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.LINK_TEXT, 'Log in'))
    )
    login_button.click()

    # Wait for the login form fields
    id_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'login-username'))
    )
    id_input.send_keys('sejin_kwon@naver.com')

    password_input = driver.find_element(By.ID, 'login-password')
    password_input.send_keys('qykfab-5reZqu-pafhug')

    login_button = driver.find_element(By.ID, 'login-button')
    login_button.click()

    # Wait for login to complete
    WebDriverWait(driver, 10).until(
        EC.url_contains('charts.spotify.com/charts')
    )

    # CSV File Download
    for country in countries:
        for date in dates:
            driver.get(f"https://charts.spotify.com/charts/view/regional-{country}-daily/{date}")
            time.sleep(2)
            # Wait for the download button to be clickable
            csv_download_button = WebDriverWait(driver, 60).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@aria-labelledby='csv_download']"))
            )
            csv_download_button.click()

In [27]:
import os
import glob
import pandas as pd


folder_path = '../data/spotify_global_200w/'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))


combined_df = pd.read_csv(csv_files[0])
combined_df.to_csv('../data/spotify_global_200w/combined.csv', index=False)


for file in csv_files[1:]:
    df = pd.read_csv(file)
    df.to_csv('../data/spotify_global_200w/combined.csv', mode='a', header=False, index=False)

In [ ]:
csv_file1 = pd.read_csv('../data/spotify_global_200w/combined.csv')
csv_file2 = pd.read_csv('../data/spotify_us_200w/combined.csv')
csv_file3 = pd.read_csv('../data/spotify_kr_200w/combined.csv')

df = pd.concat([csv_file1, csv_file2, csv_file3], ignore_index=True)

In [30]:
df.columns

Index(['rank', 'uri', 'artist_names', 'track_name', 'source', 'peak_rank',
       'previous_rank', 'days_on_chart', 'streams'],
      dtype='object')

In [38]:
song_names = df[['uri', 'artist_names', 'track_name']]

song_names.loc[:, 'id'] = song_names['uri'].str.split(':').str[-1]

song_names.drop(columns='uri', inplace=True)

song_names.drop_duplicates(subset='id', keep='first', inplace=True)
song_names.to_csv('../data/new_songs.csv', index=False)

/var/folders/8x/wdbjc6g52tvgsbjbvq4__4x00000gn/T/ipykernel_63808/1811786550.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_names.loc[:, 'id'] = song_names['uri'].str.split(':').str[-1]
/var/folders/8x/wdbjc6g52tvgsbjbvq4__4x00000gn/T/ipykernel_63808/1811786550.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_names.drop(columns='uri', inplace=True)
/var/folders/8x/wdbjc6g52tvgsbjbvq4__4x00000gn/T/ipykernel_63808/1811786550.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docu